In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
# import PIL
import tensorflow as tf
import pickle5 as pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
from random import shuffle
import datetime

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [7]:
# import pathlib
# data_dir = pathlib.Path('E:/Users/sentinel_industry/downloaded_aois/')


In [8]:
# x_size = 230
# y_size = 230
# ch1_mean, ch1_std = 970.4162, 1051.8454
# ch2_mean, ch2_std = 1176.249, 1031.262
# ch3_mean, ch3_std = 1273.2376, 1117.068
# ch4_mean, ch4_std = 2266.9050, 1241.5509
# means = [ch1_mean,ch2_mean,ch3_mean,ch4_mean]
# vari = [ch1_std, ch2_std, ch3_std, ch4_std]

# def load_features(name):
#     decoded = name.decode("UTF-8")
#     if os.path.exists(decoded):
#         with open(decoded, 'rb') as f:
#             file = pickle.load(f)
#             label = tf.strings.split(tf.strings.split(name, '/')[-1], '\\')[-2]
#             if label == 'coal':
#                 label = [1,0,0]
#             elif label == 'steel':
#                 label = [0,1,0]
#             else: label = [0,0,1]
#             if (file["B02"].shape[1]<230 or file["B02"].shape[2]<230): 
#                 print("oh oh, downloaded patch too smol:", decoded)
#             B02, B03, B08 = file['B02'][0][0:x_size,0:y_size], file['B03'][0][0:x_size,0:y_size],file['B08'][0][0:x_size,0:y_size]
#             B02 = (B02-ch1_mean)/ch1_std
#             B03 = (B03-ch2_mean)/ch2_std
#             # B04 = (B04-ch3_mean)/ch3_std
#             B08 = (B08-ch4_mean)/ch4_std
#             features = np.array([ B02, B03, B08]).transpose(1,2,0)
#             # features = np.expand_dims(features, axis=0)
#             return features, label
#             # I have commented the line below but this should return
#             # the features and the label in a one hot vector
#             # return file['features'], file['targets']
#     else:
#         print("Something went wrong!")
#         exit(-1)
        
# def data_loader(filename):
#     features, labels = tf.numpy_function(load_features, [filename], [tf.double, tf.int32])
#     # features.set_shape((None, 242,242,4))
#     # labels.set_shape(( 1))
#     return features, labels

In [3]:

ch1_mean, ch1_std = 970.4162, 1051.8454
ch2_mean, ch2_std = 1176.249, 1031.262
ch3_mean, ch3_std = 1273.2376, 1117.068
ch4_mean, ch4_std = 2266.9050, 1241.5509
means = [ch1_mean,ch2_mean,ch3_mean,ch4_mean]
vari = [ch1_std, ch2_std, ch3_std, ch4_std]

def load_features(name):
    decoded = name.decode("UTF-8")
    if os.path.exists(decoded):
        with open(decoded, 'rb') as f:
            file = pickle.load(f)
            label = tf.strings.split(tf.strings.split(name, '/')[-1], '\\')[-2]
            if label == 'coal':
                label = [1,0,0]
            elif label == 'steel':
                label = [0,1,0]
            else: label = [0,0,1]
            if (file["B02"].shape[1]<230 or file["B02"].shape[2]<230): 
                print("oh oh, downloaded patch too smol:", decoded)
            B02, B03, B04, B08 = file['B02'][0][0:x_size,0:y_size], file['B03'][0][0:x_size,0:y_size], file['B04'][0][0:x_size,0:y_size],file['B08'][0][0:x_size,0:y_size]
            B02 = (B02-ch1_mean)/ch1_std
            B03 = (B03-ch2_mean)/ch2_std
            B04 = (B04-ch3_mean)/ch3_std
            B08 = (B08-ch4_mean)/ch4_std
            features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
            # features = np.expand_dims(features, axis=0)
            return features, label
            # I have commented the line below but this should return
            # the features and the label in a one hot vector
            # return file['features'], file['targets']
    else:
        print("Something went wrong!")
        exit(-1)
        
def data_loader(filename):
    features, labels = tf.numpy_function(load_features, [filename], [tf.double, tf.int32])
    # features.set_shape((None, 242,242,4))
    # labels.set_shape(( 1))
    return features, labels

In [4]:
path = 'E:/Users/sentinel_industry/downloaded_aois/coal/'

pkl_files = glob.glob((path+"*.pickle"))


In [5]:
print(len(pkl_files))

2238


In [6]:
path_steel = 'E:/Users/sentinel_industry/downloaded_aois/steel/'
pkl_files_steel = glob.glob((path_steel+"*.pickle"))

In [7]:
path_steel_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/steel/no_ind/'
pkl_files_steel_no_ind = glob.glob((path_steel_no_ind+"*.pickle"))

In [8]:
path_coal_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/coal/no_ind/'
pkl_files_coal_no_ind = glob.glob((path_coal_no_ind+"*.pickle"))

In [9]:
print(len(pkl_files_steel))

1707


In [10]:
pkl_files = pkl_files + pkl_files_steel + pkl_files_steel_no_ind + pkl_files_coal_no_ind
shuffle(pkl_files)
# pkl_files=pkl_files[0:100]

In [16]:
# from itertools import chain

# pkl_files = list(chain.from_iterable(zip(pkl_files, pkl_files_steel)))


In [17]:
print(len(pkl_files))

7966


In [18]:
dataset_len=len(pkl_files)

In [19]:
dataset = tf.data.Dataset.from_tensor_slices((pkl_files))

def data_loader(filename):
    features, labels = tf.numpy_function(load_features, [filename], [tf.double, tf.int32])
    # features.set_shape((None, 242,242,4))
    # labels.set_shape(( 1))
    return features, labels

dataset = dataset.map(data_loader)
               # load_features, [filename], [tf.uint16, tf.string])))

In [18]:
dataset_len=len(pkl_files)

train_size = int(np.floor(0.5 * dataset_len))
valid_size = int(np.floor(0.5 * dataset_len))
test_size = int(np.floor(0.0 * dataset_len))

train = tf.data.Dataset.from_tensor_slices(pkl_files[0:train_size]).map(data_loader,num_parallel_calls=tf.data.AUTOTUNE)
valid = tf.data.Dataset.from_tensor_slices(pkl_files[train_size:train_size+valid_size]).map(data_loader,num_parallel_calls=tf.data.AUTOTUNE)
test = tf.data.Dataset.from_tensor_slices(pkl_files[train_size+valid_size:]).map(data_loader,num_parallel_calls=tf.data.AUTOTUNE)

In [19]:
# train = train.shuffle(100,  reshuffle_each_iteration=True, )
valid = valid.shuffle(100,  reshuffle_each_iteration=True, )


In [88]:
# dataset = dataset.map((
#   lambda x, y: (resize_and_rescale(x), y)))

In [13]:
for image_batch, labels_batch in dataset:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

NameError: name 'dataset' is not defined

In [90]:
train

<ShuffleDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float64, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [92]:
# ch1_mean, ch1_std = 970.4162903392252, 1051.8454702994397
# ch2_mean, ch2_std = 1176.249944943864, 1031.262597521539
# # ch3_mean, ch3_std = 1273.2376818390699, 1117.068939085472
# ch4_mean, ch4_std = 2266.9050137827385, 1241.5509461215606
# norm_layer=keras.layers.Normalization(mean=[ch1_mean, ch2_mean, ch4_mean], variance=[ch1_std**2, ch2_std**2, ch4_std**2])

In [14]:
x_size = 230
y_size = 230
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_size,y_size,3)),  # here, maybe try larger values
    layers.CenterCrop(140,140, ),
    layers.RandomCrop(120,120, seed=in_seed, ),  # to be changed with centercrop for data augmentation/ use centre first, then random?
    
    # layers.Rescaling(1./10000),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
    # norm_layer,
])

In [95]:
# model = tf.keras.Sequential([
#     resize_rescale_augment,
#     layers.Conv2D(16, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(32, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(3)
# ])
model = tf.keras.Sequential([
    resize_rescale_augment,
    # layers.Conv2D(3, 5, padding='same', activation='relu'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        pooling=None ,
        classes=3,),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/new/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.03)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [96]:
model.build()

In [97]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 120, 120, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_1 (Dense)             (None, 3)                 98307     
                                                                 
Total params: 23,686,019
Trainable params: 23,632,899
Non-trainable params: 53,120
_________________________________________________________________


In [26]:
%load_ext tensorboard

In [27]:
# !env LD_LIBRARY_PATH="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# os.environ['LD_LIBRARY_PATH']="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# os.environ['LD_INCLUDE_PATH']="C:\Anaconda3_64\pkgs\cudatoolkit-11.7.0-ha6f8bbd_10\Library\bin"
# # !set LD_INCLUDE_PATH=:/usr/local/cuda/include:/usr/local/cuda/extras/CUPTI/include

In [28]:
!env LD_LIBRARY_PATH

env: â€˜LD_LIBRARY_PATHâ€™: No such file or directory


In [29]:
%tensorboard --logdir='E:/Users/sentinel_industry/logs/train/new'

Reusing TensorBoard on port 6006 (pid 22096), started 5:48:34 ago. (Use '!kill 22096' to kill it.)

In [98]:
# now with 3ch only
epochs=100
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=1,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)

Epoch 1/100
318/318 [==============================] - 197s 579ms/step - loss: 3.5475 - categorical_accuracy: 0.4912 - val_loss: 2.0789 - val_categorical_accuracy: 0.5000 - lr: 0.0010
Epoch 2/100
318/318 [==============================] - 125s 388ms/step - loss: 1.2336 - categorical_accuracy: 0.6714 - val_loss: 2.0053 - val_categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 3/100
318/318 [==============================] - 123s 382ms/step - loss: 0.9123 - categorical_accuracy: 0.7000 - val_loss: 99.5648 - val_categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 4/100
318/318 [==============================] - 128s 396ms/step - loss: 0.9099 - categorical_accuracy: 0.6932 - val_loss: 0.6429 - val_categorical_accuracy: 0.7500 - lr: 0.0010
Epoch 5/100
318/318 [==============================] - 123s 382ms/step - loss: 1.1077 - categorical_accuracy: 0.6263 - val_loss: 3.8628 - val_categorical_accuracy: 0.6500 - lr: 0.0010
Epoch 6/100
318/318 [==============================] - 123s 383ms/step - loss: 

In [15]:
# valid = tf.data.experimental.load('savedata_valid')
# train = tf.data.experimental.load('savedata_train')
# train = train.shuffle(100,  reshuffle_each_iteration=True)

x_size = 230
y_size = 230
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_size,y_size,4)),  # here, maybe try larger values
    layers.CenterCrop(140,140, ),
    layers.RandomCrop(120,120, seed=in_seed, ),  # to be changed with centercrop for data augmentation/ use centre first, then random?
    
    # layers.Rescaling(1./10000),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
])

model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(3, 5, padding='same', activation='tanh'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False,
        weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        pooling=max ,
        classes=3,),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/threemod/resnet50v2_firsth" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.03)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.build()
model.summary()

# now with normalisation
epochs=150
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=10,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 120, 120, 4)       0         
                                                                 
 conv2d (Conv2D)             (None, 120, 120, 3)       303       
                                                                 
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 3)                 98307     
                                                                 
Total params: 23,663,410
Trainable params: 23,617,970
Non-trainable params: 45,440
_________________________________________________________________
Epoch 1/150
99/99 [==================

In [16]:
model.save('saved_models/resnet50v2_firsth')

INFO:tensorflow:Assets written to: saved_models/resnet50v2_firsth\assets


INFO:tensorflow:Assets written to: saved_models/resnet50v2_firsth\assets


In [20]:
# valid = tf.data.experimental.load('savedata_valid')
# train = tf.data.experimental.load('savedata_train')
# train = train.shuffle(100,  reshuffle_each_iteration=True)

x_size = 230
y_size = 230
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_size,y_size,4)),  # here, maybe try larger values
    layers.CenterCrop(140,140, ),
    layers.RandomCrop(120,120, seed=in_seed, ),  # to be changed with centercrop for data augmentation/ use centre first, then random?
    
    # layers.Rescaling(1./10000),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
])

model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(3, 5, padding='same', activation='tanh'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.resnet_v2.ResNet50V2(
        include_top=False,
        weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        pooling=max ,
        classes=3,),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/threemod/resnet50v2_firsth" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.03)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.build()
model.summary()

# now with normalisation
epochs=150
batch_size = 40
history = model.fit(
    valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=10,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)
model.save('saved_models/resnet50v2_secondh')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 120, 120, 4)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 120, 120, 3)       303       
                                                                 
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 flatten_2 (Flatten)         (None, 32768)             0         
                                                                 
 dense_2 (Dense)             (None, 3)                 98307     
                                                                 
Total params: 23,663,410
Trainable params: 23,617,970
Non-trainable params: 45,440
_________________________________________________________________
Epoch 1/150
99/99 [==================

KeyboardInterrupt: 

In [21]:
# model.save('saved_models/resnet50v2_secondh')

INFO:tensorflow:Assets written to: saved_models/resnet50v2_secondh\assets


INFO:tensorflow:Assets written to: saved_models/resnet50v2_secondh\assets


In [105]:
model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(3, 5, padding='same', activation='softmax'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        pooling=None ,
        classes=3,),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/threemod/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.03)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.build()
model.summary()

# now with normalisation
epochs=230
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=10,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_14 (Sequential)  (None, 120, 120, 4)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 120, 120, 3)       303       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_8 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 3)                 98307     
                                                                 
Total params: 23,686,322
Trainable params: 23,633,202
Non-trainable params: 53,120
_________________________________________________________________
Epoch 1/230
159/159 [===============

In [106]:
model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(3, 5, padding='same', activation='tanh'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights=None, # if I don't use the pre trained weights from image net, does it matter that i don't use the preprocessing step which reorders RGB to BGR and zero-centers wrt imagenet?
        input_shape=(120, 120, 3),
        pooling=None ,
        classes=3,),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/threemod/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.03)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.build()
model.summary()

# now with normalisation
epochs=240
batch_size = 40
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=10,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback]
)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_14 (Sequential)  (None, 120, 120, 4)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 120, 120, 3)       303       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_9 (Flatten)         (None, 32768)             0         
                                                                 
 dense_9 (Dense)             (None, 3)                 98307     
                                                                 
Total params: 23,686,322
Trainable params: 23,633,202
Non-trainable params: 53,120
_________________________________________________________________
Epoch 1/240
159/159 [===============

KeyboardInterrupt: 

In [32]:
import matplotlib.pyplot as plt
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(18, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


KeyError: 'binary_accuracy'

In [ ]:
iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    next_element = iterator.get_next()
    im = resize_rescale_augment(next_element[0])
    plt.imshow(np.flip(im[0,:,:,0:3], axis=2))
    plt.title(convert_onehot(next_element))

In [ ]:
def convert_onehot(next_element):
    if (next_element[1].numpy() == np.array([0, 0, 1])).all(): return 'no industry'
    elif (next_element[1].numpy() == np.array([0, 1, 0])).all(): return 'steel'
    elif (next_element[1].numpy() == np.array([1, 0, 0])).all(): return 'coal'
    else: return 'not valid'

In [ ]:
next_element[1].numpy().all()

In [ ]:
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("module.name", "/path/to/file.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = foo
spec.loader.exec_module(foo)

In [ ]:
! PATH